In [ ]:
import meteostat
import pandas as pd
import datetime

In [ ]:
stations_df = meteostat.Stations().region(country='AE').fetch().dropna()
stations_df = stations_df.drop(columns=['wmo', 'icao', 'monthly_start', 'monthly_end']).reset_index()
stations_df

In [ ]:
today = datetime.datetime.today()
stations_id = stations_df['id'].to_list()
start = datetime.datetime(2015, 1, 1, 00, 00)
end = datetime.datetime(today.year, today.month, today.day, today.hour, today.minute)

In [ ]:
df = meteostat.Hourly(stations_id, start=start, end=end, model=False) # Do not include "modelled data" - Only ones measured by weather stations.
df = df.fetch().reset_index().rename(columns={'station' : 'id'})
df = stations_df[['id', 'name']].merge(df, how='right', on='id').drop(columns=['id'])
df = df[['name', 'time', 'temp', 'rhum', 'wspd']].rename(columns = {'name' : 'weather_station', 'time' : 'datetime', 'temp' : 'temperature', 'rhum' : 'humidity', 'wspd' : 'wind_speed'})
df = pd.melt(df, id_vars=['datetime', 'weather_station'], value_vars=['temperature', 'humidity', 'wind_speed'], var_name='weather_variable', value_name='nominal')
df['granularity'] = 'hourly'
df

In [ ]:
import matplotlib.pyplot as plt

for value in ['temperature', 'humidity', 'wind_speed']:
  df[df['weather_variable'] == value].pivot(values='nominal', index='datetime', columns='weather_station').plot(figsize=(15,5), alpha=0.2, title=value)
  plt.show()

In [ ]:
normalise_dict = {'Lima International Airport': ['Lima I.A.', 'Airports']}
df['station_id'] = df['weather_station'].apply(lambda x: normalise_dict[x][1])
df['weather_station'] = df['weather_station'].apply(lambda x: normalise_dict[x][0])
df

In [ ]:
df2 = meteostat.Daily(stations_id, start=start, end=end, model=False) # Do not include "modelled data" - Only ones measured by weather stations.
df2 = df2.fetch().reset_index().rename(columns={'station' : 'id'})
df2 = stations_df[['id', 'name']].merge(df2, how='right', on='id').drop(columns=['id'])
df2 = df2[['name', 'time', 'prcp']].rename(columns = {'name' : 'weather_station', 'time' : 'datetime', 'prcp' : 'rain'})
df2 = pd.melt(df2, id_vars=['datetime', 'weather_station'], value_vars=['rain'], var_name='weather_variable', value_name='nominal')
df2['granularity'] = 'daily'
df2['station_id'] = df2['weather_station'].apply(lambda x: normalise_dict[x][1])
df2['weather_station'] = df2['weather_station'].apply(lambda x: normalise_dict[x][0])
df2

In [ ]:
df2.pivot_table(index='datetime', values='nominal', columns='weather_station', aggfunc='sum').plot(figsize=(15,5))